In [1]:
import pandas as pd
import numpy as np
import warnings

In [2]:
warnings.filterwarnings('ignore')

In [3]:
germany = pd.read_csv('germany.csv')
germany.drop(columns=['country'],inplace = True)

In [4]:
germany.query('manufacturer=="mercedes-benz"')

price    year   manufacturer    model  cylinders fuel  odometer  \
8970     1300.0  2000.0  mercedes-benz  s-class        8.0  gas  168000.0   
8971     6200.0  2000.0  mercedes-benz  s-class        8.0  gas  147000.0   
8972     2400.0  2000.0  mercedes-benz  s-class        8.0  gas  275000.0   
8973     4300.0  2000.0  mercedes-benz  s-class        8.0  gas  174975.0   
8974     9000.0  2000.0  mercedes-benz  s-class        8.0  gas   83500.0   
...         ...     ...            ...      ...        ...  ...       ...   
13517   96250.0  2021.0  mercedes-benz  s-class        8.0  gas    8774.0   
13518   89774.0  2021.0  mercedes-benz  s-class        8.0  gas   31359.0   
13519   97989.0  2022.0  mercedes-benz  s-class        8.0  gas   15591.0   
13520  110000.0  2022.0  mercedes-benz  s-class        8.0  gas    2930.0   
13521  109995.0  2022.0  mercedes-benz  s-class        8.0  gas   10424.0   

      title_status transmission   type  engine_size   hp  
8970         clean    automatic  sedan          4.3  275  
8971         clean    automatic  sedan          4.3  275  
8972         clean    automatic  sedan          4.3  275  
8973         clean    automatic  sedan          4.3  275  
8974         clean    automatic  sedan          4.3  275  
...            ...          ...    ...          ...  ...  
13517        clean    automatic  sedan          4.0  496  
13518        clean    automatic  sedan          4.0  496  
13519        clean    automatic  sedan          4.0  496  
13520        clean    automatic  sedan          4.0  496  
13521        clean    automatic  sedan          4.0  496  

[4552 rows x 12 columns]

In [5]:
germany.model = [model.lower() for model in germany.model]

In [6]:
germany.title_status = np.where(germany.title_status=="parts only","salvage",germany.title_status)

In [7]:
lables = ['missing','lien','salvage','rebuilt','clean']
nominal = np.arange(0,5,step = 1)
germany.title_status = germany.title_status.replace(lables,nominal)
germany.title_status = germany.title_status.astype(np.int32)

In [8]:
germany.odometer = np.sqrt(germany.odometer)
germany.price = np.sqrt(germany.price)

In [9]:
germany.transmission = np.where(germany.transmission=="automatic",1,0)

In [10]:
germany = pd.get_dummies(germany,prefix="",prefix_sep="")

In [11]:
X = germany.drop(columns=['price']).values
y = germany.price.values

In [12]:
from sklearn.model_selection import train_test_split,cross_val_score

In [13]:
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size = 0.3,random_state=42)

In [14]:
from sklearn.linear_model import LinearRegression

In [15]:
lm =  LinearRegression().fit(X_train,y_train)

In [16]:
lm.score(X_train,y_train)

0.8854181715058753

In [17]:
lm.score(X_test,y_test)

0.8864033306690214

In [18]:
cross_val_score(lm,X_test,y_test)

array([0.89025696, 0.88643976, 0.88080807, 0.87412375, 0.8811311 ])

In [19]:
columns = germany.drop(columns=['price']).columns

In [26]:
def predict(manufacturer,body,model,year,odometer,cyl,engine,fuel,transmission,hp,title_status):
    zeros = np.zeros(len(columns))
    title_status_dict = {'missing':0,'lien':1,'salvage':2,'rebuilt':3,'clean':4}
    zeros[0] = year
    zeros[1] = cyl
    zeros[2] = np.sqrt(odometer)
    zeros[3] = title_status_dict[title_status]
    zeros[4] = np.where(transmission=="automatic",1,0)
    zeros[5] = engine
    zeros[6] = hp



    manufacturer_idx = np.where(manufacturer == columns)[0][0]
    fuel_idx = np.where(fuel == columns)[0][0]
    type_body_idx = np.where(body == columns)[0][0]
    model_idx = np.where(model == columns)[0][0]


    if manufacturer_idx >= 0:
        zeros[manufacturer_idx] = 1


    if fuel_idx >=0:
        zeros[fuel_idx] = 1


    if type_body_idx>=0:
        zeros[type_body_idx] = 1
        
    if model_idx >=0:
        zeros[model_idx] = 1



    zeros = np.asarray([zeros])
    
    pred = lm.predict(zeros)


    return pred**2

In [28]:
predict('mercedes-benz','sedan','c-class',2023,6044,8,4.0,'gas','automatic',496,'clean')

array([57450.58286515])